In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [50]:
df_clientes = pd.read_csv('../../dados/credit_score/df_clientes_variavel_target.csv')

# Train Test Split

In [8]:
SEED = 42

In [19]:
df_train, df_test = train_test_split(df_clientes, test_size=0.25, random_state=SEED)

In [15]:
print(df_train.shape, df_test.shape)

(21307, 16) (7103, 16)


In [16]:
df_train.to_csv('../../dados/credit_score/df_clientes_train.csv', index=False)
df_test.to_csv('../../dados/credit_score/df_clientes_test.csv', index=False)

# Criação pipeline

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from imblearn.over_sampling import SMOTE

## Drop ID_Cliente

In [51]:
class DropFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, feature_to_drop='ID_Cliente') -> None:
        self.feature_to_drop = feature_to_drop

    def fit(self, df):
        return self
    
    def transform(self, df):
        if self.feature_to_drop in df.columns:
            drop_df = df.drop(columns=[self.feature_to_drop])
            return drop_df
        else:
            print(f'Variável {self.feature_to_drop} não encontrada no dataframe')
            return df

In [52]:
new = DropFeatures().transform(df_clientes)

In [54]:
new

,Tem_carro,Tem_casa_propria,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Idade,Anos_empregado,Tamanho_familia,Rendimento_anual,Categoria_de_renda,Grau_escolaridade,Estado_civil,Moradia,Ocupacao,Risco_de_credito
0,1,1,0,0,0,59,3,2,112500.0,Empregado,Ensino médio,Casado,Casa/apartamento próprio,Segurança,0
1,0,1,0,1,1,52,8,1,270000.0,Associado comercial,Ensino médio,Solteiro,Casa/apartamento próprio,Associado comercial,0
2,0,1,0,1,1,52,8,1,270000.0,Associado comercial,Ensino médio,Solteiro,Casa/apartamento próprio,Associado comercial,0
3,0,1,0,0,0,62,0,1,283500.0,Pensionista,Ensino superior,Divorciado,Casa/apartamento próprio,Pensionista,0
4,0,1,0,0,0,62,0,1,283500.0,Pensionista,Ensino superior,Divorciado,Casa/apartamento próprio,Pensionista,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28405,1,1,0,0,0,47,7,2,315000.0,Empregado,Ensino médio,Casado,Casa/apartamento próprio,Gerência,1
28406,0,1,0,1,1,34,4,2,157500.0,Associado comercial,Ensino superior,Casado,Casa/apartamento próprio,Associado comercial,1
28407,0,1,0,1,1,34,4,2,157500.0,Pensionista,Ensino superior,Casado,Casa/apartamento próprio,Pensionista,1
28408,0,1,0,0,0,49,2,2,283500.0,Empregado,Ensino médio,Casado,Casa/apartamento próprio,Vendas,1


In [48]:
new

,Tem_carro,Tem_casa_propria,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Idade,Anos_empregado,Tamanho_familia,Rendimento_anual,Categoria_de_renda,Grau_escolaridade,Estado_civil,Moradia,Ocupacao,Risco_de_credito
0,1,1,0,0,0,59,3,2,112500.0,Empregado,Ensino médio,Casado,Casa/apartamento próprio,Segurança,0
1,0,1,0,1,1,52,8,1,270000.0,Associado comercial,Ensino médio,Solteiro,Casa/apartamento próprio,Associado comercial,0
2,0,1,0,1,1,52,8,1,270000.0,Associado comercial,Ensino médio,Solteiro,Casa/apartamento próprio,Associado comercial,0
3,0,1,0,0,0,62,0,1,283500.0,Pensionista,Ensino superior,Divorciado,Casa/apartamento próprio,Pensionista,0
4,0,1,0,0,0,62,0,1,283500.0,Pensionista,Ensino superior,Divorciado,Casa/apartamento próprio,Pensionista,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28405,1,1,0,0,0,47,7,2,315000.0,Empregado,Ensino médio,Casado,Casa/apartamento próprio,Gerência,1
28406,0,1,0,1,1,34,4,2,157500.0,Associado comercial,Ensino superior,Casado,Casa/apartamento próprio,Associado comercial,1
28407,0,1,0,1,1,34,4,2,157500.0,Pensionista,Ensino superior,Casado,Casa/apartamento próprio,Pensionista,1
28408,0,1,0,0,0,49,2,2,283500.0,Empregado,Ensino médio,Casado,Casa/apartamento próprio,Vendas,1
